In [1]:
import numpy as np
import pandas as pd
import healpy as hp
import os
from DS_Planck_Unet import *
from DS_healpix_fragmentation import radec2pix
from matplotlib import pyplot as plt
from DS_data_transformation import draw_df
from DS_detector import *
from tqdm.notebook import tqdm
import re
from DS_data_transformation import get_prm

In [2]:
models = {#'pz20' : load_planck_model('/home/rt2122/Models/pz_only/pz_only.ep0020-vl0.006524-l0.003684.hdf5'),
        #'pz25': load_planck_model('/home/rt2122/Models/pz_only/pz_only.ep0025-vl0.006676-l0.003428.hdf5'),
        #'pz_act20' : load_planck_model('/home/rt2122/Models/act_ros/act_ros.ep0020-vl0.038030-l0.025285.hdf5'),
         'pz_act6' : load_planck_model('/home/rt2122/Models/act_ros/act_ros.ep0006-vl0.033158-l0.033525.hdf5')}

In [3]:
thr = 0.1

In [4]:
for model_name in models:
    #cur_cat = []
    for i in tqdm(range(48)):
        all_dict = gen_pics_for_detection(i, models[model_name], step=8)
        coords = detect_clusters_connected(all_dict, thr, i)
        #cur_cat.append(coords)
        coords.to_csv('/home/rt2122/Data/detected_cats/tmp/p{pix}_{model}_thr0.1_step8'.format(pix=i, 
                                                                                    model=model_name),
                     index=False)
    #cur_cat = pd.concat(cur_cat, ignore_index=True)
    #cur_cat.to_csv(
    #    '/home/rt2122/Data/detected_cats/full_{model}_thr{thr}_step8.csv'.format(
    #        model=model_name, thr=thr), index=False)

/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
'pz_act25' in 'pz_act20_dfasjkfha'

False

In [6]:
files = next(os.walk('/home/rt2122/Data/detected_cats/tmp/'))[-1]
len(files)

48

In [7]:
pix = [int(get_prm('p', name)) for name in files]
for i in range(48):
    if not (i in pix):
        print(i)

In [8]:
full_df = [pd.read_csv(os.path.join('/home/rt2122/Data/detected_cats/tmp/', name)) for name in files]
full_df = pd.concat(full_df, ignore_index=True)

In [9]:
full_df.shape

(22500, 13)

In [13]:
full_df = full_df.drop(['Unnamed: 0'], axis='columns')
full_df

RA        DEC  area   min_rad   max_rad  mean_rad  min_pred  \
0      288.310691  10.263022   8.0  0.482738  3.643752  1.851784  0.117075   
1      290.160115   1.404166  16.0  1.338526  3.831315  2.389915  0.100982   
2      288.123350  10.433811   3.0  0.351760  1.738945  1.139314  0.201384   
3      290.046669   1.389174   2.0  0.484719  1.815510  1.094504  0.110045   
4      287.972115  10.189269  30.0  1.932570  4.594723  3.198335  0.103385   
...           ...        ...   ...       ...       ...       ...       ...   
22495  211.604201  12.658333   NaN       NaN       NaN       NaN       NaN   
22496  211.772854  10.806414   NaN       NaN       NaN       NaN       NaN   
22497  211.775000   7.795815   NaN       NaN       NaN       NaN       NaN   
22498  211.792919   8.887455   NaN       NaN       NaN       NaN       NaN   
22499  212.654096  12.225000   NaN       NaN       NaN       NaN       NaN   

       max_pred  tRA  tDEC status catalog  
0      0.986539  NaN   NaN     fp     NaN  
1      0.154774  NaN   NaN     fp     NaN  
2      0.929666  NaN   NaN     fp     NaN  
3      0.116983  NaN   NaN     fp     NaN  
4      0.999664  NaN   NaN     fp     NaN  
...         ...  ...   ...    ...     ...  
22495       NaN  NaN   NaN     fn     act  
22496       NaN  NaN   NaN     fn     act  
22497       NaN  NaN   NaN     fn     act  
22498       NaN  NaN   NaN     fn     act  
22499       NaN  NaN   NaN     fn     act  

[22500 rows x 12 columns]

In [14]:
full_df.to_csv('/home/rt2122/Data/detected_cats/full_pz_act6_thr0.1_step8.csv', index=False)

In [11]:
df = pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_act6_thr0.1_step8.csv')
df.head()

Unnamed: 0          RA        DEC  area   min_rad   max_rad  mean_rad  \
0           0  288.310691  10.263022   8.0  0.482738  3.643752  1.851784   
1           1  290.160115   1.404166  16.0  1.338526  3.831315  2.389915   
2           2  288.123350  10.433811   3.0  0.351760  1.738945  1.139314   
3           3  290.046669   1.389174   2.0  0.484719  1.815510  1.094504   
4           4  287.972115  10.189269  30.0  1.932570  4.594723  3.198335   

   min_pred  max_pred  tRA  tDEC status catalog  
0  0.117075  0.986539  NaN   NaN     fp     NaN  
1  0.100982  0.154774  NaN   NaN     fp     NaN  
2  0.201384  0.929666  NaN   NaN     fp     NaN  
3  0.110045  0.116983  NaN   NaN     fp     NaN  
4  0.103385  0.999664  NaN   NaN     fp     NaN